In [1]:
from collections import Counter

import pandas as pd
import numpy as np

from lol_fandom import SITE, set_default_delay
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)
# set_default_delay(0.5)

In [2]:
def get_new_id(ids):
    if 'team_id' in ids.columns:
        id_list = sorted(ids['team_id'].unique())
    else:
        id_list = sorted(ids['player_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

## Teams

In [3]:
teams = pd.read_csv('./csv/teams_id.csv')
names = []

# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    scoreboard_games = pd.read_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv')
    print(f'{year} - tournament {tournaments.shape[0]}')
    print(f'{year} - scoreboard games {scoreboard_games.shape[0]}')
    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        sg = scoreboard_games.loc[scoreboard_games['OverviewPage'] == page]
        team_names = sg[['Team1', 'Team2']].unstack().unique()
        names = []
        for name in team_names:
            if name not in teams['team'].values:
                names.append(name)
        if len(names) > 0:
            names = sorted(names)
            break
    if len(names) > 0:
        print(f'https://lol.fandom.com/wiki/{page.replace(" ", "_")}')
        print(f'{page}\n{names}')
        break
if len(names) == 0:
    print('\nCompleted')

2023 - tournament 10
2023 - scoreboard games 230
	LLA/2023 Season/Opening Promotion
	LPL/2023 Season/Spring Season
	LCK/2023 Season/Spring Season
	LEC/2023 Season/Winter Season
	CBLOL/2023 Season/Split 1
	LCO/2023 Season/Split 1 Stage 1
	LLA/2023 Season/Opening Season
	TCL/2023 Season/Winter Season
	LCS/2023 Season/Spring Season
	LJL/2023 Season/Spring Season

Completed


In [ ]:
teams.loc[teams['team'].str.contains(
    'rainbow',
    case=False
)]

In [ ]:
if len(names) > 0:
    teams = pd.concat(
        [
            teams,
            pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
            # pd.Series({'team': names.pop(0), 'team_id': 323}).to_frame().T
        ],
        ignore_index=True
    )
names

In [ ]:
teams = teams.sort_values(by=['team_id', 'team']).reset_index(drop=True)
teams

In [ ]:
teams.to_csv('./csv/teams_id.csv', index=False)

## Players

In [4]:
players = pd.read_csv('./csv/players_id.csv')
names = {}

# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    scoreboard_games = pd.read_csv(
        f'./csv/scoreboard_games/{year}_scoreboard_games.csv'
    )
    scoreboard_players = pd.read_csv(
        f'./csv/scoreboard_players/{year}_scoreboard_players.csv'
    )
    print(year)
    print(f'{year} - {tournaments.shape[0]} tournaments | {scoreboard_games.shape[0]} scoreboard games | {scoreboard_players.shape[0]} scoreboard players')
    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        sp = scoreboard_players.loc[scoreboard_players['OverviewPage'] == page]
        player_names = sp['Link'].unique()
        for i, name in enumerate(player_names):
            team = sp.loc[sp['Link'] == name, 'Team'].unique()
            team = sorted(team)[0]
            position = sp.loc[sp['Link'] == name, 'Role Number'].unique()[0]
            player_names[i] = (team, position, name)
        player_names = sorted(player_names)
        for (team, _, name) in player_names:
            if name not in players['player'].values:
                lst = names.get(team, [])
                lst.append(name)
                names[team] = lst
        if len(names) > 0:
            break
    if len(names) > 0:
        print(f'{page}')
        for team, player_lst in names.items():
            print(f'{team}\n  {player_lst}')
        break
if len(names) == 0:
    print('\nCompleted')

2023
2023 - 10 tournaments | 230 scoreboard games | 2300 scoreboard players
	LLA/2023 Season/Opening Promotion
	LPL/2023 Season/Spring Season
	LCK/2023 Season/Spring Season
	LEC/2023 Season/Winter Season
	CBLOL/2023 Season/Split 1
	LCO/2023 Season/Split 1 Stage 1
	LLA/2023 Season/Opening Season
	TCL/2023 Season/Winter Season
	LCS/2023 Season/Spring Season
	LJL/2023 Season/Spring Season

Completed


In [ ]:
print(f'https://lol.fandom.com/wiki/{page.replace(" ", "_")}')
print(page)
teams = list(names.keys())
print(teams)
print()
print(teams[0])
print(names[teams[0]])

In [ ]:
players.loc[players['player'].str.contains(
    'ace..ko',
    case=False
)]
# players.loc[players['player_id'] == 135]

In [ ]:
if len(names) > 0:
    name = names[teams[0]].pop(0)
    players = pd.concat(
        [
            players,
            pd.Series({'player': name, 'player_id': get_new_id(players)}).to_frame().T
            # pd.Series({'player': name, 'player_id': 2967}).to_frame().T
        ],
        ignore_index=True
    )
    if len(names[teams[0]]) == 0:
        del names[teams[0]]
        del teams[0]
print(teams)
print()
if len(teams) > 0:
    print(teams[0])
    print(names[teams[0]])

In [ ]:
players = players.sort_values(by=['player_id', 'player']).reset_index(drop=True)
players

In [ ]:
while True:
    players['lower'] = players['player'].str.lower()
    lst = sorted(list(players[['lower', 'player_id']].itertuples(index=False)))

    is_complete = True
    prev = lst[0]
    for cur in lst[1:]:
        if prev[0] == cur[0] and prev[1] != cur[1] and cur[1] != 1869:
            is_complete = False
            break
        prev = cur

    if not is_complete:
        print('incomplete')
        candidate = players.loc[players['lower'] == prev[0]]
        print(candidate)
        df = players.loc[players['player_id'].isin(candidate['player_id'].values)]
        if candidate.shape[0] != df.shape[0]:
            print('improper id')
            print(df)
            break
        else:
            players.loc[players['lower'] == prev[0], 'player_id'] = min(players.loc[players['lower'] == prev[0], 'player_id'])
            print('after correction')
            print(players.loc[players['lower'] == prev[0]])
            print('\n')
    else:
        print('completed')
        break

In [ ]:
is_complete = True

counter = Counter(players[['player', 'player_id']].itertuples(index=False))
lst = [key for key, value in counter.items() if value > 1]
if len(lst) > 0:
    is_complete = False
    print('incomplete')
    print(lst)
    for (name, id) in lst:
        idx = players.loc[players['player'] == name].index
        players.drop(idx[1:], inplace=True)
else:
    print('completed')

In [ ]:
if is_complete:
    players[['player', 'player_id']].to_csv('./csv/players_id.csv', index=False)
    print('Save complete')